In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import gc
import tensorflow as tf
import pandas as pd
import numpy as np
import re, string
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

import logging
logger = logging.getLogger('wikitokeniser')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr = logging.FileHandler('tokenizer.log')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr)
logger.setLevel(logging.INFO)
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
nltk.download('omw-1.4')
lemma = WordNetLemmatizer()

2023-03-11 06:42:34.317860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 06:42:34.916390: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-11 06:42:34.916419: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-11 06:42:36.337319: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
chunks = pd.read_csv('traindata.out', delimiter='\t', 
                     chunksize= 500000, header=None,
                     names=['wikiid', 'token', 'label'],error_bad_lines=False)

In [3]:
def filter_stopwords(line):
    linef = ""
    splitted = str(line).split(" ")
    for w in splitted:
        if w not in stop_words:
            linef = linef+" "+w
    return linef
            
    

In [4]:

def process_sentence(line):
    line = str(line)
    linef = ""
    line = re.sub(r"[^A-Za-z0-9]"," ", line)
    #line = re.sub(r"[^p{L}]"," ", line)
    line = re.sub(r"^\W*","", line)
    line = re.sub(r"$\W*","", line)
    line = re.sub(r"\W+"," ", line)
    splitted = str(line).split(" ")
    for w in splitted:
        if w not in stop_words:
            word_lemma = lemma.lemmatize(w)
            linef = linef+" "+word_lemma
    return linef

In [ ]:
def isWikiId1(line):
    line = re.sub(r"Q[0-9]+","",str(line))
    if len(line) == 0:
        return "1"
    else:
        return "0"

In [5]:
def isWikiId(line):
    line = re.sub(r"Q[0-9]+","",str(line))
    if len(line) == 0:
        return "1"
    else:
        line = re.sub(r"^\W*","", line)
        line = re.sub(r"(nan)","", line)
        if len(line) == 0:
            return "2"
    return "0"
        


In [ ]:
#gc.enable()
while(True):
    try:
        df = next(chunks)
        if df.shape[0] < 1:
            break;
        #df['rank'] = df.groupby('wikiid')['wikiid'].rank('first')
        df['isWiki'] = df.token.map(lambda x : isWikiId(x))
        red_df = df[df['isWiki'] == "0"]
        red_df['filtered_token'] = red_df.token.map(lambda x : process_sentence(x))
        new_df = red_df[['wikiid', 'filtered_token']]
        wiki_df = df[df['isWiki'] == "1"]
        wiki_df = wiki_df[['wikiid', 'token']]
        new_df.to_csv('filtered.csv', mode='a', index=False, header=False)
        wiki_df.to_csv('wikirelation.csv', mode='a', index=False, header=False)
        #gc.collect()
    except Exception as e:
        logger.error('Failed to parse this block: '+ str(e))
        continue
    

In [ ]:
df2 = pd.read_csv('filtered.csv',header=None,names=['wikiid', 'token'], chunksize= 1000000,error_bad_lines=False)
desired_vocab_size = 20000000
t = tf.keras.preprocessing.text.Tokenizer(num_words=desired_vocab_size)
gc.enable()

In [ ]:
i = 0
while(True):
    i += 1
    
    try:
        wikidf = next(df2)
        if wikidf.shape[0] < 1:
            break;
        wikidf['wikiid'] = wikidf['wikiid'].astype(str)
        wikidf['token'] = wikidf['token'].astype(str)
        wikidf['token_len'] = wikidf.token.map(lambda x : len(str(x).split(" ")))
        wikidf_large = wikidf[wikidf['token_len'] > 3]
        wikidf_small = wikidf[wikidf['token_len'] < 4]
        t.fit_on_texts(wikidf_small['token'])
        wikidf_small.to_csv('wikismall.csv', mode='a', index=False, header=False)
        gc.collect()
        print(i)
    except:
        print(len(t.word_index))
        wikidf_small.to_csv('wikismall.csv', mode='a', index=False, header=False)
        continue
        

In [ ]:
t.word_index

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(wikidf['token'], wikidf['wikiid'], test_size = 0.3, random_state = 42)

In [ ]:
wiki_relation = pd.read_csv('wikirelation.csv', names=['wikiid','relid'])

In [20]:
df3 = pd.read_csv('wikirelation.csv',header=None,names=['wikiid', 'relid'], chunksize= 1000000,error_bad_lines=False)


In [22]:
wiki_relation = next(df3)
wiki_rel = wiki_relation[wiki_relation.wikiid != wiki_relation.relid]
wiki_id_feat = wiki_rel.groupby('wikiid')['relid'].apply(concat_labels)
wiki_id_feat.to_csv('wikifeature.csv', mode='a', index=False, header=False)
wiki_id_feat.shape

(474391,)

,wikiid,relid
0,P108,Q18608871 Q26935994
1,P109,Q18608871 Q18610173
2,P111,Q21077852
3,P119,Q18608756 Q18635217 Q22983697 Q51122237
4,P131,Q18615777 Q18647519 Q18647515 Q70564278
...,...,...
474386,Q9014707,Q4167836
474387,Q9014714,Q4167836
474388,Q9014722,Q4167836
474389,Q9014729,Q11266439


In [ ]:
def concat_labels(x):
    line = ""
    try:
        for q in x:
            line = line+" "+q
    except:
        line = "QERROR"
    return line

df3 = pd.read_csv('wikirelation.csv',header=None,names=['wikiid', 'relid'], chunksize= 1000000,error_bad_lines=False)

i = 0
while(True):
    i += 1
    
    try:
        wiki_relation = next(df3)
        if wiki_relation.shape[0] < 1:
            break;
        wiki_rel = wiki_relation[wiki_relation.wikiid != wiki_relation.relid]
        wiki_id_feat = wiki_rel.groupby('wikiid')['relid'].apply(concat_labels)
        wiki_id_feat = wiki_id_feat.to_frame().reset_index()
        wiki_id_feat.to_csv('wikifeature.csv', mode='a', index=False, header=False)
        print(i)
    except:
        print('error = '+i)
        continue

In [14]:
wiki_relation.shape

(1000000, 2)

In [5]:
wiki_rel = wiki_relation[wiki_relation.wikiid != wiki_relation.relid]

In [8]:
def concat_labels(x):
    line = ""
    for q in x:
        line = line+" "+q
    return line

In [15]:
wiki_id_feat = wiki_rel.groupby('wikiid')['relid'].apply(concat_labels)

In [16]:
wiki_id_feat

wikiid
P102                                               Q18608871
P19         Q18608756 Q18635217 Q26935994 Q18608871 Q9956...
P22                  Q22964231 Q26935994 Q99567026 Q18647519
P39                             Q18608871 Q26935994 Q7810129
P88                                                Q18618644
                                 ...                        
Q999975                                                   Q5
Q99998                                               Q849778
Q999982                                                   Q5
Q999997                                                   Q5
Q999998                                                   Q5
Name: relid, Length: 459879, dtype: object

In [1]:
indexed = pd.read_csv('wikifeature.csv',header=None,names=['wikiid', 'wikilabels'], chunksize= 100000,error_bad_lines=False)


NameError: name 'pd' is not defined

In [5]:
df = next(indexed)
df

,wikiid,wikilabels
0,P102,Q18608871
1,P19,Q18608756 Q18635217 Q26935994 Q18608871 Q9956...
2,P22,Q22964231 Q26935994 Q99567026 Q18647519
3,P39,Q18608871 Q26935994 Q7810129
4,P88,Q18618644
...,...,...
99995,Q1820958,Q5
99996,Q1820960,Q4167410
99997,Q1820962,Q8502
99998,Q182097,Q19832486


In [3]:
!pip install dask

In [4]:
import dask.dataframe as dd

In [11]:
wiki_indexed = dd.read_csv("indexed.csv",names=['wikiid', 'wiki_ind_sent'],error_bad_lines=False)

In [12]:
wiki_feature = dd.read_csv('wikifeature.csv', names=['wikiid', 'wikilabel'],error_bad_lines=False)

In [ ]:
combined = dd.merge(wiki_feature, wiki_indexed, on ='wikiid', how='left').compute()